In [ ]:
import tensorflow as tf

In [ ]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import shutil
import numpy as np
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for file in glob.glob("/content/drive/MyDrive/Vehicle_Detection/Data/*xml"):
  os.remove(file)

In [ ]:
os.getcwd()
%cd /content/sample_data/Extract

/content/sample_data/Extract


In [ ]:
!unzip /content/drive/MyDrive/Dataset/archive.zip

In [ ]:
# Moving XML file to an annotation folder
for file in glob.glob("/content/sample_data/Extract/train/Final Train Dataset/*xml"): 
    src_path = os.path.abspath(file)
    dst_path = "/content/drive/MyDrive/Vehicle_Detection/Data/xml_annotations/" + os.path.basename(file)
    shutil.move(src_path, dst_path)

In [ ]:
# Splitting the dataset
# First split into images and annotation separately
# Split Images
# From image names annotation names can be puled

source = "/content/drive/MyDrive/Vehicle_Detection/Data/Final Train Dataset"
rootdir = "/content/drive/MyDrive/Vehicle_Detection/Data"
allFileNames = os.listdir(source)
np.random.shuffle(allFileNames)

test_ratio = 0.20

train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                      [int(len(allFileNames)* (1 - test_ratio))])

train_FileNames = [source+'/'+ name for name in train_FileNames.tolist()]

test_FileNames = [source+'/' + name for name in test_FileNames.tolist()]

for name in train_FileNames:
  shutil.copy(name, rootdir +'/Train_Data/' + os.path.basename(name) )

for name in test_FileNames:
  shutil.copy(name, rootdir +'/Test_Data/' +  os.path.basename(name))

In [ ]:
# Moving Annotation XML file to Images Folder

for filename in os.listdir("/content/drive/MyDrive/Vehicle_Detection/Data/Train_Data"):
  src = "/content/drive/MyDrive/Vehicle_Detection/Data/xml_annotations/" + os.path.splitext(filename)[0] +".xml"
  dst = "/content/drive/MyDrive/Vehicle_Detection/Data/Train_Data/" + os.path.splitext(filename)[0] +".xml"
  shutil.copy(src,dst)

In [ ]:
# Changing XML filename

import glob,os
for file in glob.glob("/content/drive/MyDrive/Vehicle_Detection/Data/Test_Data/*xml"):
  path = os.path.abspath(file)
  basename = os.path.basename(file)
  tree = ET.parse(path)
  root = tree.getroot()
  xmlfilename = os.path.splitext(basename)[0]
  ext_check = os.path.splitext(xml_i)[1]
  
  if (os.path.exists("/content/drive/MyDrive/Vehicle_Detection/Data/Test_Data/" + xmlfilename +  '.jpg')):
    ext = 'jpg' 
  if (os.path.exists("/content/drive/MyDrive/Vehicle_Detection/Data/Test_Data/" + xmlfilename +  '.jpeg')):
    ext = 'jpeg'
  if (os.path.exists("/content/drive/MyDrive/Vehicle_Detection/Data/Test_Data/" + xmlfilename +  '.png')):
    ext = 'png'
  if (os.path.exists("/content/drive/MyDrive/Vehicle_Detection/Data/Test_Data/" + xmlfilename +  '.JPG')):
    ext = 'JPG'
  if (os.path.exists("/content/drive/MyDrive/Vehicle_Detection/Data/Test_Data/" + xmlfilename +  '.PNG')):
    ext = 'PNG'
  root.find('filename').text = xmlfilename + '.' + ext
  tree.write(path)

In [ ]:
# Created a folder Vehicle detection with subfolders Dataset and Training
# Created an Annotation with xml file and Images with images

In [ ]:
# Cloning Tensorflow repository to /content
%cd /content
!git clone --q https://github.com/tensorflow/models.git

/content


In [ ]:
%cd /content/models/research

# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/models/research
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.9 MB 17.4 MB/s 
     |████████████████████████████████| 352 kB 43.7 MB/s 
     |████████████████████████████████| 2.1 MB 43.6 MB/s 
     |████████████████████████████████| 25.9 MB 1.9 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 511.8 MB 7.7 kB/s 
     |████████████████████████████████| 1.1 MB 33.9 MB/s 
     |████████████████████████████████

In [ ]:
# Testing Tensorflow API Installation
!python object_detection/builders/model_builder_tf2_test.py

2022-09-08 14:33:48.171393: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Running tests under Python 3.7.13: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0908 14:33:49.030614 140368381704064 model_builder.py:1102] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 1.53s
I0908 14:33:49.707009 140368381704064 test_util.py:2459] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 1.53s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 1.13s
I0908 14:33:50.840762 140368381704064 test_util.py:2459] time(__main__.ModelBuilderTF2Test.test_cr

In [ ]:
# Saving tf record inside Annotation
# Create train data:
!python /content/drive/MyDrive/Vehicle_Detection/generate_tfrecord.py -x /content/drive/MyDrive/Vehicle_Detection/Data/Train_Data -l /content/drive/MyDrive/Vehicle_Detection/label_map.pbtxt -o /content/drive/MyDrive/Vehicle_Detection/Data/Annotation/train.record

# Create test data:
!python /content/drive/MyDrive/Vehicle_Detection/generate_tfrecord.py -x /content/drive/MyDrive/Vehicle_Detection/Data/Test_Data -l /content/drive/MyDrive/Vehicle_Detection/label_map.pbtxt -o /content/drive/MyDrive/Vehicle_Detection/Data/Annotation/test.record


2022-09-08 14:36:08.288345: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecord file: /content/drive/MyDrive/Vehicle_Detection/Data/Annotation/train.record
2022-09-08 14:37:03.810937: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecord file: /content/drive/MyDrive/Vehicle_Detection/Data/Annotation/test.record
